In [ ]:
# import all relevant libraries
import pandas as pd
import numpy as np
import uuid
import re
import json
from pandas.plotting import register_matplotlib_converters
from pandas import json_normalize

In [2]:
charging = pd.read_csv("charging_sessions.csv", encoding = "ISO-8859-1")

In [3]:
charging.head()

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


In [4]:
charging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


In [ ]:
charging = charging.drop(columns=['Unnamed: 0']).reset_index(drop=True)

charging['connectionTime'] = pd.to_datetime(charging['connectionTime'])
charging['disconnectTime'] = pd.to_datetime(charging['disconnectTime'])
charging['doneChargingTime'] = pd.to_datetime(charging['doneChargingTime'])


#id sind floats in CSV aber sind alle ints
#null werte zu 0.0 umwandeln
charging['userID'] = charging['userID'].fillna(0.0)
charging['userID'] = charging['userID'].astype(int)

charging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                66450 non-null  object             
 1   connectionTime    66450 non-null  datetime64[ns, UTC]
 2   disconnectTime    66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime  62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered      66450 non-null  float64            
 5   sessionID         66450 non-null  object             
 6   siteID            66450 non-null  int64              
 7   spaceID           66450 non-null  object             
 8   stationID         66450 non-null  object             
 9   timezone          66450 non-null  object             
 10  userID            66450 non-null  int32              
 11  userInputs        49187 non-null  object             
dtypes: datetime64[ns, UTC](3), float64(1), int32(1), int64(1), o

In [6]:
def parse_json(x):
    if isinstance(x, str):
        x = x.replace("'", '"').strip()
        x = re.sub(r'\bTrue\b', 'true', x)
        x = re.sub(r'\bFalse\b', 'false', x)    
        try:
            return json.loads(x)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e} | Input: {x}")  # Print error and input
            return []  # Return an empty list if parsing fails
    return []

charging['userInputs'] = charging['userInputs'].apply(parse_json)
user_inputs_expanded = json_normalize(charging['userInputs'].explode()).add_prefix('userInputs_')
charging = charging.drop(columns=['userInputs']).reset_index(drop=True).join(user_inputs_expanded.reset_index(drop=True))

charging['userInputs_userID'] = charging['userID'].astype(int)

charging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   id                             66450 non-null  object             
 1   connectionTime                 66450 non-null  datetime64[ns, UTC]
 2   disconnectTime                 66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime               62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered                   66450 non-null  float64            
 5   sessionID                      66450 non-null  object             
 6   siteID                         66450 non-null  int64              
 7   spaceID                        66450 non-null  object             
 8   stationID                      66450 non-null  object             
 9   timezone                       66450 non-null  object             
 10  userID                

In [7]:
#check if userInputs_userID always equal to userID
# 
charging['userID_matches'] = charging['userID'] == charging['userInputs_userID']
charging['userID_matches'].describe()

count     66450
unique        1
top        True
freq      66450
Name: userID_matches, dtype: object

In [8]:
charging = charging.drop(columns=['userID_matches']).reset_index(drop=True)
charging = charging.drop(columns=['userInputs_userID']).reset_index(drop=True)
charging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   id                             66450 non-null  object             
 1   connectionTime                 66450 non-null  datetime64[ns, UTC]
 2   disconnectTime                 66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime               62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered                   66450 non-null  float64            
 5   sessionID                      66450 non-null  object             
 6   siteID                         66450 non-null  int64              
 7   spaceID                        66450 non-null  object             
 8   stationID                      66450 non-null  object             
 9   timezone                       66450 non-null  object             
 10  userID                

In [9]:
charging['connectionDay'] = charging['connectionTime'].dt.date
charging['disconnectionDay'] = charging['disconnectTime'].dt.date

charging['date_matches'] = charging['connectionDay'] == charging['disconnectionDay']
charging['date_matches'].describe()


count     66450
unique        2
top        True
freq      38361
Name: date_matches, dtype: object

*not all loading processes start and end on the same day*

proposed solution: split charging processes on 2 different dates into 2 with uniform distribution of kWhDelivered and first start time + 0:00 and second 0:00 + end time
**this is only necessary when checking charging processes based on day**

In [10]:
charging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   id                             66450 non-null  object             
 1   connectionTime                 66450 non-null  datetime64[ns, UTC]
 2   disconnectTime                 66450 non-null  datetime64[ns, UTC]
 3   doneChargingTime               62362 non-null  datetime64[ns, UTC]
 4   kWhDelivered                   66450 non-null  float64            
 5   sessionID                      66450 non-null  object             
 6   siteID                         66450 non-null  int64              
 7   spaceID                        66450 non-null  object             
 8   stationID                      66450 non-null  object             
 9   timezone                       66450 non-null  object             
 10  userID                

In [11]:
print(charging.isnull().sum())

id                                   0
connectionTime                       0
disconnectTime                       0
doneChargingTime                  4088
kWhDelivered                         0
sessionID                            0
siteID                               0
spaceID                              0
stationID                            0
timezone                             0
userID                               0
userInputs_WhPerMile             16867
userInputs_kWhRequested          16867
userInputs_milesRequested        16867
userInputs_minutesAvailable      16867
userInputs_modifiedAt            16867
userInputs_paymentRequired       16867
userInputs_requestedDeparture    16867
connectionDay                        0
disconnectionDay                     0
date_matches                         0
dtype: int64


doneChargingTime == NULL could be considered as a car not being charged the entire requested kWh 

In [12]:
unique_stations = charging["stationID"].nunique()
print(unique_stations)

107


In [13]:
unique_parking_spaces = charging["spaceID"].nunique()
print(unique_parking_spaces)

107


In [14]:
unique_sites = charging["siteID"].nunique()
print(unique_sites)

2


2 sites with 107 overall charging stations

In [15]:
timezone_unique_values = charging['timezone'].nunique()
print("Number of different timezone values:", timezone_unique_values)


Number of different timezone values: 1


As we can see, the timezone value for all variables is exactly the same. This doesn't help us for the anlayisis, therefore we can drop this column

In [ ]:
#insert here code for dropping column 'timezone'
charging = charging.drop(columns=['timezone'])
charging.head()


,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,userID,userInputs_WhPerMile,userInputs_kWhRequested,userInputs_milesRequested,userInputs_minutesAvailable,userInputs_modifiedAt,userInputs_paymentRequired,userInputs_requestedDeparture,connectionDay,disconnectionDay,date_matches
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,194,250.0,25.0,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT",2020-01-02,2020-01-02,True
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,4275,280.0,70.0,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT",2020-01-02,2020-01-02,True
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,344,400.0,8.0,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT",2020-01-02,2020-01-03,False
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,1117,400.0,8.0,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT",2020-01-02,2020-01-02,True
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,334,400.0,8.0,20.0,65.0,"Thu, 02 Jan 2020 14:00:19 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT",2020-01-02,2020-01-02,True


chargingTime could later be used to show how many people leave their cars plugged in after they are fully charged, blocking the station for other potential customers and causing higher demand.

Computing the number of null rows for chargingTime:

In [ ]:
# Compute all non-null rows of the dataset
total_rows = len(charging)

# Count non-null values of the column doneCharging time
non_null_values_doneCharging = charging['doneChargingTime'].count()

# Count null values by subtracting non-null count from total rows
null_count = total_rows - non_null_values_doneCharging

# Calculate the percentage of null values
null_percentage = (null_count / total_rows) * 100

print(f'Non-null rows: {non_null_values_doneCharging}')
print(f'Null rows: {null_count}')
print(f'Percentage of null values: {null_percentage:.2f}%')


Non-null rows: 62362
Null rows: 4088
Percentage of null values: 6.15%


-UserID and userInput same amount of null values 
->implies missing user ID corresponds to missing userInput

-doneCharging could be either set to disconnectTime or splitting data in disconnectTime==null and disconnectTime!=null

-multiple userInputs how to handle -> taking only most recent input?